## 2.6 Iterative Methods


Algorithms based on Gaussian elimination are called **exact** or, more properly, **direct**
methods because they would generate exact solutions for the system of linear equations $Ax = b$
after a finite number of operations, if not for rounding error. Such methods are ideal
for moderately-sized linear equations, but may be impractical for large ones. Other
methods, called **iterative methods** can often be used to solve large linear system of equations
more efficiently if the A matrix is sparse, that is, if $A$ is composed mostly of zero
entries. Iterative methods are designed to generate a sequence of increasingly accurate
approximations to the solution of a system of linear equations, but generally do not yield an exact
solution after a prescribed number of steps, even in theory.


The most widely-used iterative methods for solving a linear equation $Ax = b$ are
developed by choosing an easily invertible matrix $Q$ and writing the system
in the equivalent form

$$Q x = b + (Q - A)x$$

or

$$x = Q^{-1}b + (I - Q^{-1}A)x.$$

This form of the system suggests the iteration rule



$$x^{(k+1)} \leftarrow   Q^{-1}b + (I - Q^{-1}A)x^{(k)}$$

Ideally, the so-called splitting matrix Q will satisfy two criteria. First, $Q^{-1}b$ and
$Q^{-1}A$ should be relatively easy to compute. This is true if Q is either diagonal or
triangular. Second, the iterates should converge quickly to the true solution of the
system. If

$$||I - Q^{-1}A|| < 1$$

in any matrix norm, then the iteration rule is a contraction mapping and is guaranteed
to converge to the solution of the system from any initial value.

The two most popular iterative methods are the **Gauss-Jacobi** and **Gauss-Seidel**
methods. 

 - The Gauss-Jacobi method sets $Q$ equal to the diagonal matrix formed from the diagonal entries of A. 


 - The Gauss-Seidel method sets $Q$ equal to the upper triangular matrix formed from the upper triangular elements of A.

## Gauss-Jacobi Method

Here, the user specifies the data A and b and an initial guess $x$ for the solution of
the linear system of equations, typically the zero vector or $b$. Iteration continues until the norm
of the change dx in the iterate falls below the specified convergence tolerance tol or
until a specified maximum number of allowable iterations maxit are performed.

In [1]:
import numpy as np
from numpy.linalg import inv # NumPy Linear Algebra Library
import scipy
import scipy.linalg   # SciPy Linear Algebra Library

In [2]:
#Let's consider this example

#Assign the appropriate values to the matrix of coefficients A
A = np.array([[54, 14, -11, 2],
               [14, 50, -4, 29],
               [-11, -4, 55, 22],
               [2, 29, 22, 95]])
print("Left hand side of linear system, matrix A:\n", A)

Left hand side of linear system, matrix A:
 [[ 54  14 -11   2]
 [ 14  50  -4  29]
 [-11  -4  55  22]
 [  2  29  22  95]]


In [3]:
#Assign the appropriate values to the vector of constants b
b = np.array([1, 1, 1, 1])

print("Right hand side of linear equation system, vector b:\n",b)

Right hand side of linear equation system, vector b:
 [1 1 1 1]


In [4]:
def gjacobi(A, b, maxit, tol, x0=None):
    """
    Solve a system of linear equations with the Gauss-Jacobi iterative method outlined in the book.
    It relies on the updating eq:
        x = inv(Q)(b - Ax)
    Where Q is the diagonal matrix obtained from A
    """
    # If we have not provided an initial array for x make a new one
    if x0==None:
        x = np.array([1 for _ in np.arange(A.shape[1])])
    else:
        x = x0
        
    # Get the diagonal matrix from A (np.diag(A) would extract a vector)
    Q = np.diag(np.diag(A))
    
    # Get the inverse of Q
    # Note: we could write the formula for the inverse of Q directly
    Qinv = inv(Q)
        
    # Begin the iterations, updating the vector x
    for it in np.arange(maxit):
        dx =  Qinv.dot(b - A.dot(x))
        x = x + dx
        #We are using the Frobenius norm
        #https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.norm.html
        if np.linalg.norm(dx)<tol:
        #If we wanted to rely on the Sup norm, we would use the following 
        #if np.linalg.norm(dx,np.inf)<tol:
            break

    print("x:\n", x)
    print("it:\n", it)

In [5]:
#Call the Gauss-Jacobi routine
gjacobi(A, b, 1000, 1e-08, x0=None)

x:
 [ 0.01893441  0.01680508  0.02335523 -0.00041086]
it:
 30


Check the result with a direct method (notice that they are not exactly identical):

In [6]:
np.linalg.solve(A,b)

array([ 0.01893441,  0.01680508,  0.02335523, -0.00041085])

In [7]:
def gseidel(A, b, maxit, tol, lamb, x0=None):
    """
    Solve a system of linear equations with the Gauss-Seidel iterative method outlined in the book.
    It relies on the updating eq:
        x = lambda*inv(Q)(b - Ax)
    Where Q is the upper triangular matrix obtained from A    
    """
    # If we have not provided an initial array for x make a new one
    if x0 == None:
        x = np.array([1 for _ in np.arange(A.shape[1])])
    else:
        x = x0
    
    # Get the upper triangular matrix from A
    Qtmp1 = np.triu(A)
    # Get the diagonal matrix from A
    Qtmp2 = np.diag(np.diag(A))
    # Get the Gauss-Seidel update matrix
    Q = Qtmp1 + Qtmp2
    
    # We could get the lower triangular matrix from A and it would still work
    #Q = np.tril(A)
    
    # Get the inverse of Q
    Qinv = inv(Q)
    
    # Begin the iterations, updating the vector x
    for it in np.arange(maxit):
        dx = Qinv.dot(b -A.dot(x))
        #Note: we include a relaxation weight to possibly make the updates less drastic
        x = x + lamb*dx
        if np.linalg.norm(dx)<tol:
            break
            
    print("x:\n", x)
    print("it:\n", it)

In [8]:
#Call the Gauss-Seidel routine (neglecting the relaxation parameter, as it is set to 1)
gseidel(A, b, 1000, 1e-08, 1, x0=None)

x:
 [ 0.01893441  0.0168051   0.02335524 -0.00041087]
it:
 54


In [9]:
#Call the Gauss-Seidel routine (setting the relaxation parameter to 0.1)
gseidel(A, b, 1000, 1e-08, 0.1, x0=None)

x:
 [ 0.01893441  0.0168051   0.02335525 -0.00041088]
it:
 611


In [10]:
#Call the Gauss-Seidel routine (setting the relaxation parameter to 2.0)
gseidel(A, b, 1000, 1e-08, 2.0, x0=None)

x:
 [ 0.01893441  0.01680509  0.02335524 -0.00041087]
it:
 22
